In [3]:
# import all modules, some might not be necessary (DIE NOG VERWIJDEREN)
from ema_workbench import (
    Model,
    MultiprocessingEvaluator,
    ScalarOutcome,
    IntegerParameter,
    optimize,
    Scenario,
    Constraint,
)
from ema_workbench.em_framework.optimization import EpsilonProgress, epsilon_nondominated
from ema_workbench.util import ema_logging

from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem, ArchiveLogger

# from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd


In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

In [ ]:
model, steps = get_model_for_problem_formulation('A4 Only')
problem = to_problem(model, searchover="levers")

In [6]:
# load the selected scenarios
scenarios_df = pd.read_csv('results/selected_scenarios.csv')
scenarios_df.rename(columns={'Unnamed: 0': 'scenario_id'}, inplace=True)
scenarios_df = scenarios_df.set_index('scenario_id')
scenarios_df

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,composite_ooi
scenario_id,,,,,,,,,,,,,,,,,,,,
37056,37,147.594102,1.0,0.000002,286.853435,1.0,0.038971,289.758018,1.0,0.404188,68.176412,1.5,0.255461,279.044680,1.5,0.226114,3.5,3.5,3.5,2.305843
4380,14,191.473309,1.5,0.000030,171.209130,10.0,0.114888,200.050550,1.0,0.866653,48.825374,1.0,0.290939,130.256529,1.0,0.990261,4.5,1.5,1.5,1.964212
6910,95,243.430458,1.5,0.000084,191.977855,10.0,0.808600,295.936040,1.5,0.735589,191.475260,1.5,0.033379,287.858038,1.0,0.778650,1.5,2.5,3.5,1.745123
21741,11,59.837465,1.0,0.000369,241.439262,1.5,0.926252,135.423150,10.0,0.566519,211.200069,1.0,0.130396,214.298793,1.5,0.450323,2.5,3.5,2.5,1.688380
4717,104,40.525349,10.0,0.161001,187.789168,1.5,0.010373,205.979011,1.0,0.958783,340.402481,1.0,0.086792,75.494126,1.5,0.826855,1.5,1.5,1.5,3.851439


In [27]:
scenario_ids = scenarios_df.index.to_list()
archive_list = []
results_list = []
convergence_list = []
amount_of_seeds = 5

# for scenario in scenario_ids:
#     for i in range(amount_of_seeds):
#         archives = ArchiveLogger.load_archives(f"./archive/Policy_search_arch{float(scenario)}_seed{i}.tar.gz") #ToDO
#         archive_list.append(archives)

#         result = pd.read_csv(f"./results/Policy_search_scen_{float(scenario)}_seed{i}results.csv")
#         results_list.append(result)

#         convergence = pd.read_csv(f"./results/Policy_search_scen_{float(scenario)}_seed{i}convergence.csv")
#         convergence_list.append(convergence)

archive_dict = {}
results_dict = {}
convergence_dict = {}  
for i in range(len(scenario_ids)):
    archive_dict[scenario_ids[i]] = archive_list[i]
    results_dict[scenario_ids[i]] = results_list[i]
    convergence_dict[scenario_ids[i]] = convergence_list[i]

 
# ToDo misschien nog een dictionary maken maar dat doe je dan met itterrows.
    

In [ ]:
policy_dict = {}
epsilon = [100, 0.01, 100, 100, 0.01] # OF, [0.05] * len(model.outcomes)
for i in range(len(scenario_ids)):
    df = epsilon_nondominated(results_list[i], epsilon, problem)
    policy_dict[scenario_ids[i]] = df
    n_policies = df.shape[0]
    print(f"Scenario {scenario_ids[i]} has {n_policies} non-dominated policies")

In [ ]:
convergence_metrics = {}
for scenario in scenario_ids:
    convergence_metrics[scenario] = []

for scenario in scenario_ids:
    hv = HypervolumeMetric(policy_dict[scenario], problem)
    gd = GenerationalDistanceMetric(policy_dict[scenario], problem, d=1)
    ei = EpsilonIndicatorMetric(policy_dict[scenario], problem)
    ig = InvertedGenerationalDistanceMetric(policy_dict[scenario], problem, d=1)
    sm = SpacingMetric(problem)

    
    for archives in archive_list:
        metrics = []
        for nfe, archive in archives.items():
            scores = {
                "generational_distance": gd.calculate(archive),
                "hypervolume": hv.calculate(archive),
                "epsilon_indicator": ei.calculate(archive),
                "inverted_gd": ig.calculate(archive),
                "spacing": sm.calculate(archive),
                "nfe": int(nfe),
            }
            metrics.append(scores)
        metrics = pd.DataFrame.from_dict(metrics)

        # sort metrics by number of function evaluations
        metrics.sort_values(by="nfe", inplace=True)
        convergence_metrics.append(metrics)
    

In [ ]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

ax1, ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()